# Fleming Pipeline - ipython notebook version

### Note on astrometry.net 

If the solve keeps timing out and you have `Config.astrometry_timeout` as a large value (or `-1`) then check the timeout in `~/.astropy/config/astroquery.cfg`.


#### How to get a key
If you do want to upload (eg if machine takes too long solving) you will need a key and add it to `~/.astropy/config/astroquery.cfg` and `pipeline/astrometry_api_key.txt`.
The first is a config file (example layout is as below) latter is just a text file containing the key and nothing else.

[Go to the website](http://nova.astrometry.net/api_help) and sign in and follow the instructions.

Example config:
```
[astrometry_net]

api_key = XXXXX
timeout = 1200 ## 20 mins
server = http://nova.astrometry.net
```

## General imports and setup
Run this, change the config cell to suit the field that you want to run for.

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports
import pipeline
from pipeline import *
from pipeline import Pipeline

from datetime import datetime
import os

In [ ]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

## A4 paper
mpl.rcParams['figure.figsize'] = [11.3, 8.7]

In [ ]:
config = Config(
    raw_image_dir = os.path.expanduser("~/mnt/jgt/2022/0301"),
    image_prefix = "l138_0",
    bias_prefix = "bias",
    n_sets = 9,
    fits_extension = ".fits",
    fits_date_format = "%Y.%m.%dT%H:%M:%S.%f",
    has_filter_in_header = False,
    n_sample_periods = 100,
    amplitude_score_threshold = 0.85,
)

## Run the pipeline
Run using the general setup from `pipeline/Pipeline.py`.

This is as far as you need to run if you want a single field.

In [ ]:
Pipeline.run(config, show_plots=True, show_errors=False)

## Pipeline breakdown
Below is a rough copy of the `Pipeline.run()` method.

Used for picking certain parts to run/rerun.
Hopefully minimal kernel restarts are needed when running.

In [ ]:
r = Reducer(config, "No filter") ## Only "No filter" for Trius

r.reduce(skip_existing=True)

In [ ]:
c = Cataloguer(config)

catalogue_image = os.path.join(config.image_dir, config.image_format_str.format(1, 1))

In [ ]:
#n_sources = c.generate_catalogue(catalogue_image, solve=True)
#c.generate_image_times()

n_sources = c.generate_catalogue(catalogue_image, solve=False)

In [ ]:
sf = ShiftFinder(config, n_sources)

sf.generate_shifts()

In [ ]:
ff = FluxFinder(config, n_sources)

In [ ]:
ff.make_light_curves()

In [ ]:
da = DataAnalyser(config, adjusted=False)

In [ ]:
mean, std, med, n_positive = da.get_means_and_stds()
da.plot_means_and_stds()
source_ids = da.get_source_ids()

In [ ]:
vd = VariableDetector(config, source_ids, mean, std, med, n_positive, adjusted=False)
exclude_from_avg = vd.std_dev_search(config.avg_exclude_threshold)
avg_ids = da.get_ids_for_avg(exclude_ids)

In [ ]:
da.make_avg_curve(avg_ids)
ff.plot_avg_light_curve(config.avg_curve_path, show=True)

In [ ]:
ff.create_adjusted_light_curves()

In [ ]:
da = DataAnalyser(config, adjusted=True)
mean, std, med, n_positive = da.get_means_and_stds()
da.plot_means_and_stds()
source_ids = da.get_source_ids()

In [ ]:
vd = VariableDetector(config, source_ids, mean, std, med, n_positive, adjusted=True)

In [ ]:
variable_ids_s = vd.std_dev_search(config.variability_threshold)
variable_ids_a = vd.amplitude_search(config.amplitude_score_threshold)
variable_ids = variable_ids_a

In [ ]:
ff.plot_given_light_curves(variable_ids, adjusted=True, show=True, show_errors=False)

In [ ]:
results_table = da.output_results(variable_ids, vd)
ff.create_thumbnails(results_table, show=True)

## Plots

Create plots for the report


In [ ]:

## Very much not automated, need to change everything by hand

ncols = 4
fig, axs = plt.subplots(nrows=3,ncols=ncols, figsize=(16,9))
fig.tight_layout()
fig.subplots_adjust(hspace=0.4, wspace=0.2)

for i, (field_id, source_id, n_periods) in enumerate(hand_picked[36:48]):   
    config = Config(
        image_prefix = field_id,
    )
    
    ## Grab the RA and DEC of the source
    cat = Utilities.read_catalogue(config)
    idx = np.where(cat['id'] == source_id)[0][0]
    ra  = cat['RA'][idx]
    dec = cat['DEC'][idx]
    if ra > 360 or dec > 180:
        continue
    ## Grab the adjusted light curve
    lc_path = os.path.join(config.adjusted_curve_dir,
            config.source_format_str.format(source_id))
    lc = np.genfromtxt(lc_path, dtype=config.light_curve_dtype)

    ## Plot the light curve
    axs[i//ncols,i%ncols].scatter(lc['time']/3600, lc['counts'], marker="x", s=2)

    axs[i//ncols,i%ncols].set_xlabel("Time [hours]")
    axs[i//ncols,i%ncols].set_ylabel("Normalised flux [arb. u.]")
    coord = SkyCoord(ra*u.degree, dec*u.degree)
    axs[i//ncols,i%ncols].set_title("{}: {}"
                .format(config.source_format_str.format(source_id)[:-4], 
                        coord.to_string("hmsdms", precision=0)))
#fig.delaxes(axs[2,3])
#fig.delaxes(axs[2,2])
#fig.delaxes(axs[2,1])
#fig.delaxes(axs[1,3])
#fig.delaxes(axs[1,2])
#fig.delaxes(axs[1,1])
plt.savefig("hand_picked_unsure_4.pdf", bbox_inches="tight")

## Debug land

Use at your own peril.

This isn't important to the pipeline, just used to test things.
Can be deleted if needed

If you change something in the `pipeline/*.py` then you need to reload the modules differently instead of with the normal imports.

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports
import pipeline
from pipeline import *
from pipeline import Pipeline

from datetime import datetime
import os

In [ ]:
# REMOVE ME WHEN NOT USING DARK MODE
import matplotlib as mpl
COLOR = "white"
#COLOR = "black"
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

## A4 paper
mpl.rcParams['figure.figsize'] = [11.3, 8.7]

In [ ]:
config = Config(
    raw_image_dir = os.path.expanduser("~/mnt/jgt/2022/0301"),
    image_prefix = "l138_0",
    bias_prefix = "bias",
    n_sets = 9,
    fits_extension = ".fits",
    fits_date_format = "%Y.%m.%dT%H:%M:%S.%f",
    has_filter_in_header = False,
    n_sample_periods = 100,
    amplitude_score_threshold = 0.85,
)

In [ ]:
## DataAnalyser, un-adjusted
da = DataAnalyser(config, adjusted=False)

mean, std, med, n_positive = da.get_means_and_stds()
source_ids = da.get_source_ids()
da.plot_means_and_stds()
    
vd = VariableDetector(config, source_ids, mean, std, med, n_positive, adjusted=False)
exclude_ids = vd.std_dev_search(config.avg_exclude_threshold, adjusted=False)
avg_ids = da.get_ids_for_avg(exclude_ids)

da.make_avg_curve(avg_ids)
ff.plot_avg_light_curve(config.avg_curve_path, show=True)



In [ ]:
Pipeline.run_existing(config, assume_already_adjusted=True, show_plots=True, show_errors=False)

In [ ]:
    config = Config(
        raw_image_dir = os.path.expanduser("~/mnt/data/tmp/napier3/Variable"),
        image_dir = os.path.expanduser("~/mnt/data/tmp/napier3/Variable"),
        image_prefix = "Variable",
        has_filter_in_header = False,
        n_sets = 1,
        set_size = 169,
        image_width = 1663,
        image_height = 1252,
    )
    c = Cataloguer(config)

    catalogue_set_number = 1
    catalogue_image_number = 1

    catalogue_image_path = os.path.join(config.image_dir,
            config.image_format_str
            .format(catalogue_set_number, catalogue_image_number))

    n_sources = c.generate_catalogue(catalogue_image_path, solve=False, write_file=False)
    
    ff = FluxFinder(config, n_sources)
    
    ff.plot_given_light_curves([108], show=True)



In [ ]:
import astropy.io.fits as fits
from pipeline import *

config = Config(
    #raw_image_dir = os.path.expanduser("~/mnt/jgt/2022/0301"),
    image_prefix = "l198",
    n_sets = 9,
)

file = os.path.expanduser("~/mnt/data/tmp/jgt_images/r_l198_1_001.fit")
out_file = os.path.expanduser("./test.fit")

c = Cataloguer(config)

_, wcs_header = c.get_wcs_header(file)
data = fits.getdata(file, ignore_missing_end=True)
head = fits.getheader(file, ignore_missing_end=True)

In [ ]:
hdu = fits.ImageHDU(data, head)
hdu2 = fits.PrimaryHDU(header=wcs_header)
hdu.scale("uint16")
hdul = fits.HDUList([hdu2, hdu], None)
hdul.writeto(out_file, overwrite=True)

In [ ]:
new_header = fits.Header()

for item in head.items():
    new_header.append(item, end=True)

for item in wcs_header.items():
    if item[0] == "bscale" or item[0] == "bzero":
        continue
    new_header.append(item, end=True)
    
new_hdu = fits.PrimaryHDU(data=data, header=new_header)
new_hdu.scale("uint16")
hdul = fits.HDUList([new_hdu], None)
hdul.writeto(out_file, overwrite=True)